In [ ]:
from input_output import *
from scipy.optimize import curve_fit

In [ ]:
print("Available datasets:\n\t" + ',\n\t'.join(os.listdir(sims_root())))
results_name = input("Input the name of a dataset: ")
# construct a PatchySimResultSet object from the specified directory
results = choose_results(results_name)

In [ ]:
target_name = input(f"Input the name of an analysis target ({','.join(results.targets.keys())}): ")

target = results.targets[target_name]

In [ ]:
# narrow type = set of parameters specifying bonding angles between patches
nt_options = ', '.join(f"{nt}" for nt in reduce(set.intersection, [set(r.narrow_types()) for r in datasets]))
nt = int(input(f"Input the narrow type of the data ({nt_options}): "))
# temperature = int(input(f"Input a temperature (options: {','.join(results.temperatures())}):"))
# cutoff = the lower limit for counting clusters as partially assembled forms of the target
# cutoff = 1 -> only count fully assembled clusters
cutoff = float(input("Input the graph match cutoff: "))
relative = input("Calculate relative yields?") == "True"
overreach = input("Include overreaching graphs in yield calculations?") == "True"

In [ ]:
df = results.calculateYields(target_name, cutoff, overreach)

In [ ]:
df = df[df['potential'] == nt]

In [ ]:
if (relative):
    for t, yld in zip(results.temperatures(), yield_data):
        yld['yield'] = yld['yield'] / (yld['num_assemblies'] * target['rel_count'])
    ymax = 1.0
else:
    if (df['num_assemblies'].unique().size > 1):
        print ("Assembly counts are not consistant, scaling may be wonky. You may want to do a relative yield analysis.")
    ymax = df['num_assemblies'].max() * target['rel_count']

In [ ]:
model = lambda(x, a, b): a-(b/x)

In [ ]:
# popt = optimal parameters array
# pcov = covariance of parameters
popt = {}
pcov = {}
for t in results.temperatures()):
    (poptp[t], pcov[t], _, _, _) = curve_fit(model, df['time'], df['yield'])
    perr = np.sqrt(np.diag(pcov))
    df['temp'==t]["myield"] = model(df['temp'==t]['time'], *poptp[t]),
    df['temp'==t]["myield_min"] = model(xs['temp'==t]['time'], *(popt[t] - perr)),
    df['temp'==t]["myield_max"] = model(xs['temp'==t]['time'], *(popt[t] + perr))

SyntaxError: unmatched ')' (427238713.py, line 7)

In [ ]:
chart = alt.Chart(df, f"Model of {results_name} formation to {target_name} at NT={nt}")
chart = chart.mark_circle(size=60).encode(
    x=alt.X("time:Q", axis=alt.Axis(tickCount=5)),
    y=alt.Y(
        "yield:Q",
        title="Yield",
        scale=alt.Scale(
            domain=(0, ymax)
        )
    ),
    color="temp"
) + chart.mark_line().encode(
    x=alt.X("time:Q"),
    y=alt.Y("myield"),
    color="temp"
) + chart.mark_area().encode(
    x="time:Q",
    y="myield_min:Q",
    y2="myield_max:Q"
    color="temp"
)